In [1]:
from mymodule import save_bin,read_bin

In [2]:
ontology = read_bin("ontology.pickle")

## Wikidata IDからXeno cant ID一覧を取得

In [ ]:
# protected_dataにコピーを作成して守る

In [ ]:
# # id2Xeno.csvの作成
# import requests
# from bs4 import BeautifulSoup
# import csv

# with open('saved_data/id2Xeno.csv', mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file,fieldnames=["wikidata_id", "xeno_canto_id"])#, fieldnames=["wikidata_id", "xeno_canto_id"]
#     writer.writeheader()
    
#     for i in range(0,len(ontology)):
#         print('\r%d / %d' %(i, 50646), end='')
#         try:
#             url = 'https://www.wikidata.org/entity/'+ontology[i]["id"]+'.json'
#             response = requests.get(url)
#             data = response.json()
            
#             the_id = list(data["entities"].keys())[0]#リダイレクト対策
#             claims = data['entities'][the_id]['claims']
#             if 'P2426' in claims:
#                 print('\r%d / %d              ' %(i, 50646), end='')
#                 xeno_canto_id = claims['P2426'][0]['mainsnak']['datavalue']['value']
#                 writer.writerow({"wikidata_id":the_id, "xeno_canto_id":xeno_canto_id})
#             else:
#                 print('\r%d / %d  nodata' %(i, 50646), end='')
#         except:
#             pass


50645 / 50646  nodata      

## Xeno cant IDから音声データID一覧を取得

In [26]:
from XenoScraping import url2soup,soup2page_num,sound_ids
import pandas as pd

df = pd.read_csv("saved_data/id2Xeno.csv",header = None)

In [27]:
id2Xeno = dict(zip(df[0], df[1]))
del id2Xeno["wikidata_id"]
the_ids = list(id2Xeno.keys())

xeno_canto_ids = []
for k in the_ids:
    xeno_canto_ids.append(id2Xeno[k])

In [ ]:
# 空白の処理

import requests
from bs4 import BeautifulSoup
import os
from mymodule import save_bin,read_bin

Xeno2sound_ids = read_bin("saved_data/Xeno2sound_ids.bin")
Xeno2sound_ids_space = dict()
len_df = len(df)-1

for j in range(0,len_df):
    if the_ids[j] not in Xeno2sound_ids or Xeno2sound_ids[the_ids[j]] == []:
        species_query = xeno_canto_ids[j].replace(' ', '%20')# 鳥名を検索クエリに
        first_soup = url2soup(f"https://xeno-canto.org/species/"+species_query)
        
        n = soup2page_num(first_soup)
        all_sound_ids = []
        
        for i in range(1,n+1):
            print('\r%d / %d   :  %d / %d               ' %(j, len_df, i, n), end='')
            soup = url2soup("https://xeno-canto.org/species/"+species_query+"?pg="+str(i))
            all_sound_ids = all_sound_ids + sound_ids(soup)
            
        Xeno2sound_ids_space[the_ids[j]] = all_sound_ids

In [ ]:
# save_bin("saved_data/Xeno2sound_ids_space.bin",Xeno2sound_ids_space)

In [61]:
from collections import defaultdict

def concat_dict(dict1, dict2):
    result = defaultdict(list)
    for d in (dict1, dict2):
        for key, value in d.items():
            result[key].extend(value)
            result[key] = list(set(result[key]))
    result = dict(result)
    return result

In [ ]:
# data_1 = read_bin("saved_data/Xeno2sound_ids.bin")
# data_2 = read_bin("saved_data/Xeno2sound_ids_space.bin")

# Xeno2sound_ids = concat_dict(data_1,data_2)
# save_bin("saved_data/Xeno2sound_ids_all.bin",Xeno2sound_ids)
# del data_1,data_2

## 音声データのダウンロード

In [5]:
# import pandas as pd
# import os
from mymodule import save_bin,read_bin,directory_to_dict

Xeno2sound_ids = read_bin("saved_data/Xeno2sound_ids_all.bin")

# directory_path = './xeno_data'
# result = directory_to_dict(directory_path)

# df = pd.read_csv("saved_data/id2Xeno.csv",header = None)# 順番通りに取得する必要があるため
# the_ids = list(df[0])[1:]

# os.makedirs('xeno_data', exist_ok=True)

In [2]:
## データ欠損ファイル名を列挙，再処理jini 

# nodata_files = []#1039
# for the_id in the_ids:
#     if the_id in result and result[the_id] == []:
#         nodata_files.append(the_id)
# print("len:",len(nodata_files))

# lack_files = []
# for the_id in the_ids:
#     if the_id in result and the_id in Xeno2sound_ids:
#         if result[the_id] != []:
#             if len(result[the_id])>=101:
#                 pass
#             else:
#                 d = len(Xeno2sound_ids[the_id])-len(result[the_id])
#                 if d!=0:
#                     lack_files.append(the_id)
# print("len:",len(lack_files))

len: 105


In [3]:
from mymodule import save_bin,read_bin
# target_data = list(Xeno2sound_ids.keys())
# save_bin("saved_data/target_data.bin",target_data)
target_data = read_bin("saved_data/target_data.bin")

In [ ]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

# target_data = list(Xeno2sound_ids.keys())
## データ欠損ファイル名を列挙，再処理
# target_data = nodata_files#
# target_data = lack_files


def download_sound(sound_id, the_id):
    sound_num = str(sound_id[2:])  # XC削除
    # sound_path = os.path.join("xeno_data", the_id, sound_num + ".mp3")
    sound_path = os.path.join("D:","xeno_data", the_id, sound_num + ".mp3")#外付けデバイス
    
    if not os.path.exists(sound_path):
        try:
            audio_response = requests.get(f"https://xeno-canto.org/{sound_num}/download")
            with open(sound_path, 'wb') as f:
                f.write(audio_response.content)
                # print(message)
                message = f"{sound_num}.mp3 downloaded"
            
        except Exception as e:
            # print(message)
            message = f"Error downloading {sound_num}.mp3: {e}"
        return message
    else:
        message = f"{sound_num}.mp3 already exists"
        # print(message)
        return message

def download_all_sounds(j,the_id, sound_ids):
    bird_path = os.path.join("D:","xeno_data", the_id)  # 鳥種フォルダ作成
    os.makedirs(bird_path, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(download_sound, sound_id, the_id) for sound_id in sound_ids]
        i = 0
        for future in as_completed(futures):
            print('\r【%s】  %d / %d   :  %d / %d   ...%s                                     '
                  % (the_id, j, len(target_data), i, len(Xeno2sound_ids[the_id]),future.result()),
                  end='')
            i = i+ 1

def cut_over_101(the_id):
    input_list = Xeno2sound_ids[the_id]
    if len(input_list) > 101:
        input_list = input_list[:101]
    return input_list


for j in range(0,len(target_data)):
    # print(j)
    the_id = target_data[j]
    bird_path = os.path.join("D:","xeno_data", the_id)# 鳥種フォルダ作成
    os.makedirs(bird_path, exist_ok=True)

    if the_id in Xeno2sound_ids:
        ids_list = cut_over_101(the_id)
        download_all_sounds(j,the_id, ids_list)
    else:
        pass

【Q12268032】  4999 / 11019   :  33 / 34   ...274610.mp3 already exists                                        

In [7]:
len(target_data)

11019

In [ ]:
len(target_data)

11019

In [ ]:
# import requests
# import os
# import re

# for j in range(220,len(nodata_files)):
#     the_id = nodata_files[j]
#     bird_path = os.path.join("xeno_data", the_id)# 鳥種フォルダ作成
#     os.makedirs(bird_path, exist_ok=True)

#     i = 0
#     if the_id in Xeno2sound_ids:
#         s = 0
#         for sound_id in Xeno2sound_ids[the_id]:
#             if s > 100:
#                 break
#             else:
#                 try:
#                     print('\r%d / %d   :  %d / %d                  ' % (j, len(Xeno2sound_ids), i, len(Xeno2sound_ids[the_id])), end='')
#                     sound_num = str(sound_id[2:])  # XC削除
#                     sound_path = os.path.join("xeno_data", the_id, sound_num + ".mp3")
                    
#                     # ここでファイルの存在をチェック
#                     if not os.path.exists(sound_path):
#                         audio_response = requests.get("https://xeno-canto.org/" + sound_num + "/download")
#                         with open(sound_path, 'wb') as f:
#                             f.write(audio_response.content)
#                         i = i + 1
#                     else:
#                         print(f"\nFile {sound_num}.mp3 already exists, skipping download.")
                    
#                     s = s + 1
#                 except:
#                     print('\r%d / %d   :  %d / %d  error           ' %(j, len(Xeno2sound_ids), i, len(Xeno2sound_ids[the_id])), end='')
#                     error.append([the_id,sound_id])
#                     s = s + 1

9192 / 9626   :  78 / 135                   

In [2]:
# error